# Module

In [1]:
cd /home/rh/Arbeitsraum/bitbucket/share/SLMI2

/home/rh/Arbeitsraum/bitbucket/share/SLMI2


In [2]:
# From the repository
from util import *
from read_data import *
data_names = list(name2file_name.keys())
print(data_names)

# Basic modules
import os
import glob
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from graph_tool.all import *

#name = "wiod2016"
name = data_names[3]
print("We are going to use: " + name)
# Enable original_format to use the format as originally provided
data_dict = get_data(name,original_format=False)
data_dict.keys()
#df_nodes = data_dict["df_nodes"]
df_edges = data_dict["df_edges"]

['blogcatalog', 'homosapiens', 'wikipos', 'enron', 'unvote', 'untrade', 'uslegis_net', 'uslegis_net_small_dyn', 'uslegis_net_dyn', 'uslegis_hyp_dyn', 'contacts', 'dawn_net', 'dawn_hyp', 'ndc_net', 'ndc_hyp', 'coauth_dblp_net', 'coauth_dblp_hyp', 'wiod2016', 'wiod2013', 'wiodlong', 'eth', 'bitcoinalpha', 'bitcoinotc', 'uscourt']
We are going to use: enron


In [3]:
g = Graph()
use_all = 0

In [4]:
uni_nodes = list(set(df_edges["source"]))
uni_nodes.extend(list(set(df_edges["target"])))
uni_nodes = list(set(uni_nodes))  

In [5]:
node2index = dict()
for i in range(len(uni_nodes)):
    node2index.update({uni_nodes[i]:i})
    

g.add_vertex(len(uni_nodes))
weight = g.new_edge_property("double")



for i in range(len(df_edges)):
    source_index = node2index[df_edges["source"].iloc[i]]
    target_index = node2index[df_edges["target"].iloc[i]]
    e = g.add_edge(g.vertex(source_index),g.vertex(target_index))
    weight[e] = 1

g.edge_properties["weight"] = weight

# Approximation but it works well

In [6]:
%%time
state_ndc = minimize_blockmodel_dl(g, state_args=dict(deg_corr=False))
state_dc  = minimize_blockmodel_dl(g, state_args=dict(deg_corr=True))

print("Non-degree-corrected DL:\t", state_ndc.entropy())
print("Degree-corrected DL:\t", state_dc.entropy())

Non-degree-corrected DL:	 50067.446423085414
Degree-corrected DL:	 50197.47761333116
CPU times: user 56 s, sys: 435 ms, total: 56.4 s
Wall time: 8.18 s


# Refinements using merge-split MCMC

In [7]:
%%time
S1 = state_ndc.entropy()

for i in range(1000): # this should be sufficiently large
    state_ndc.multiflip_mcmc_sweep(beta=np.inf, niter=10)

S2 = state_ndc.entropy()

print("Improvement:", S2 - S1)

Improvement: -460.6297356687646
CPU times: user 22.1 s, sys: 5.81 ms, total: 22.1 s
Wall time: 22.1 s


In [8]:
%%time
# All use this
#mcmc_equilibrate(state_ndc, wait=1000, nbreaks=2, mcmc_args=dict(niter=10))

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.58 µs


In [31]:
b = state_ndc.get_blocks()
B0 = state_ndc.get_matrix()



r = b[10]   # group membership of vertex 10
print(r)

36


In [45]:

df_nodes = pd.DataFrame(node2index.items())
df_nodes.columns = ["nodes","index"]

In [50]:
out = []
for i in range(len(df_nodes)):
    ind = df_nodes["index"].iloc[i]
    out.append(b[ind])
    
df_nodes["sbm"] = out

In [53]:
df_nodes["sbm"].value_counts()

sbm
61    13
53     8
5      7
22     6
78     5
      ..
4      1
77     1
68     1
81     1
76     1
Name: count, Length: 83, dtype: int64